In [ ]:
#Vars
input_size = 256
batch_size = 16
train_size = 8000
validation_size = 1000
test_size = 1000
learning_rate = 0.001
epochs = 10
dropout = 0.0
dense_size = 1024

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import os 
import datetime

fruit = "apple"
model_current = "vgg16"

file_name = f'{model_current}_{fruit}_{batch_size}_{epochs}_{dropout}'
log_dir = f'logs/fit/{model_current}_{fruit}_{batch_size}_{epochs}_{dropout}'
run_dir = f'./{model_current}_{fruit}_{batch_size}_{epochs}_{dropout}/'
os.makedirs(run_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [ ]:
from keras.applications.vgg16 import VGG16

pre_trained_model = VGG16(
    input_shape = (input_size,input_size,3),
    include_top = False,
    weights='imagenet'
)

for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

#Flatten
x = layers.Flatten()(pre_trained_model.output)

#Fully connected layer con 1024 hidden units y ReLU
x = layers.Dense(dense_size, activation='relu')(x)
#Dropout rate
if (dropout>0):
    x = layers.Dropout(dropout)(x)
x = layers.Dense(dense_size/2, activation='relu')(x)

#Función de clasificación softmax - Clasificación binaria
x = layers.Dense(2, activation='softmax')(x)

model = Model(pre_trained_model.input,x)

model.compile(optimizer = RMSprop(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics = [tf.keras.metrics.Accuracy(name='accuracy'),
                         tf.keras.metrics.Precision(name='precision'), 
                         tf.keras.metrics.Recall(name='recall')])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.)
validation_datagen = ImageDataGenerator(rescale = 1./255.)
test_datagen = ImageDataGenerator(rescale = 1./255.)

train_dir = f'../input/fruit-defect-dataset/FRUIT_DEFECT_TRAINING_DATASET/{fruit.upper()}/TRAIN'
validation_dir = f'../input/fruit-defect-dataset/FRUIT_DEFECT_TRAINING_DATASET/{fruit.upper()}/VALIDATION'
test_dir = f'../input/fruit-defect-dataset/FRUIT_DEFECT_TRAINING_DATASET/{fruit.upper()}/TEST'

train_generator = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,shuffle=True, class_mode = 'categorical', target_size=(input_size,input_size), seed=42)
validation_generator = validation_datagen.flow_from_directory(validation_dir,batch_size=batch_size,shuffle=False, class_mode = 'categorical', target_size=(input_size,input_size))
test_generator = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,shuffle=False, class_mode = 'categorical', target_size=(input_size,input_size))

labels = train_generator.class_indices
print(labels)

In [ ]:
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
checkpoint = ModelCheckpoint(f"{run_dir}{file_name}.h5", monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callbacks_list = [checkpoint, tensorboard_callback]



history = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = train_size/batch_size,
    epochs = epochs,
    validation_steps = validation_size/batch_size,
    verbose = 1,
    callbacks = callbacks_list
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
precision = history.history['precision']
val_precision = history.history['val_precision']
recall = history.history['recall']
val_recall = history.history['val_recall']

n_epochs = range(len(acc))

plt.plot(acc)
plt.plot(val_acc)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(["Train", 'Validation'], loc = 'upper left')
plt.savefig(f"{run_dir}accuracy.jpg")
plt.figure()

plt.plot(loss)
plt.plot(val_loss)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(["Train", 'Validation'], loc = 'upper left')
plt.savefig(f"{run_dir}loss.jpg")
plt.figure()

plt.show()

In [ ]:
import csv

header = ["train_acc", "val_acc", "train_loss", "val_loss", "train_precision", "val_precision", "train_recall", "val_recall"]

with open(f'{run_dir}metrics.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    for idx in range(len(acc)):
        line = [acc[idx],val_acc[idx],loss[idx],val_loss[idx], precision[idx], val_precision[idx], recall[idx], val_recall[idx]]        
        writer.writerow(line)

In [ ]:
import pandas as pd
df = pd.read_csv(f'{run_dir}metrics.csv')
print('Best Metrics')
print(df.loc[df['val_acc'].idxmax()])

In [ ]:
results = model.evaluate(test_generator, verbose=1)
print('Test Accuracy: ', results[1])

In [ ]:
!zip -r "{file_name}_run.zip" "{run_dir}"

In [ ]:
!zip -r "{file_name}_logs.zip" "{log_dir}"

In [ ]:
from IPython.display import FileLink

FileLink(f"{file_name}_run.zip")

In [ ]:
from IPython.display import FileLink
FileLink(f"{file_name}_logs.zip")

Prediction

In [ ]:
!pip install progress

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
from progress.bar import Bar
#from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input

#fresh_dir = 'test/fresh'
#defect_dir = 'test/defect'

fresh_dir = f'../input/fruit-defect-dataset/FRUIT_DEFECT_TRAINING_DATASET/{fruit.upper()}/TEST/FRESH'
defect_dir = f'../input/fruit-defect-dataset/FRUIT_DEFECT_TRAINING_DATASET/{fruit.upper()}/TEST/DEFECT'

fresh_files = os.listdir(fresh_dir)
defect_files = os.listdir(defect_dir)

model = keras.models.load_model(f'{run_dir}{file_name}.h5')

count_fresh_right = 0
count_fresh_wrong = 0
count_defect_right = 0
count_defect_wrong = 0

bar = Bar('Processing', max=len(fresh_files)+len(defect_files))
for file in fresh_files:
    bar.next()
    img = image.load_img(fresh_dir+'/'+file, target_size=(input_size, input_size))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    classes = model.predict(preprocess_input(img_array))
    if classes[0][0]>0.5:
        count_fresh_wrong += 1
    elif classes[0][1]>0.5:
        count_fresh_right += 1

for file in defect_files:
    bar.next()
    img = image.load_img(defect_dir+'/'+file, target_size=(256, 256))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    classes = model.predict(preprocess_input(img_array))

    if classes[0][0]>0.5:
        count_defect_right += 1
    elif classes[0][1]>0.5:
        count_defect_wrong += 1

bar.finish()
print("Fresh right: ", count_fresh_right)
print("Fresh wrong: ", count_fresh_wrong)
print("Defect right: ", count_defect_right)
print("Defect wrong: ", count_defect_wrong)
print("Acc: ", count_fresh_right+count_defect_right)

Export local